In [2]:
import findspark
findspark.init()
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error
import pandas as pd
import os

#https://repo1.maven.org/maven2/com/github/housepower/clickhouse-native-jdbc/2.7.1/clickhouse-native-jdbc-2.7.1.jar
packages = [
    "com.github.housepower:clickhouse-spark-runtime-3.4_2.12:0.7.3",
    "com.clickhouse:clickhouse-jdbc:0.6.0-patch4",
    "com.clickhouse:clickhouse-http-client:0.6.0-patch4",
    "org.apache.httpcomponents.client5:httpclient5:5.3.1",
    "com.github.housepower:clickhouse-native-jdbc:2.7.1"
]
ram = 12
cpu = 12*3
# Define the application name and setup session
appName = "Connect To ClickHouse via PySpark"
spark = (SparkSession.builder
         .appName(appName)
         # .config("spark.jars.packages", "com.github.housepower:clickhouse-spark-runtime-3.4_2.12:0.7.3,com.clickhouse:clickhouse-jdbc:0.6.0,com.clickhouse:clickhouse-http-client:0.6.0,org.apache.httpcomponents.client5:httpclient5:5.3.1,com.github.housepower:clickhouse-native-jdbc:2.7.1")
         .config("spark.jars.packages", ','.join(packages))
         .config("spark.sql.catalog.clickhouse", "xenon.clickhouse.ClickHouseCatalog")
         # .config("spark.sql.catalog.clickhouse", "xenon.clickhouse.ClickHouseCatalog")
         .config("spark.sql.catalog.clickhouse.host", "34.118.21.172") # from GCP VM instances or other server ip
         .config("spark.sql.catalog.clickhouse.protocol", "http")
         .config("spark.sql.catalog.clickhouse.http_port", "8123")
         .config("spark.sql.catalog.clickhouse.user", "default")
         .config("spark.sql.catalog.clickhouse.password", "1234")
         .config("spark.sql.catalog.clickhouse.database", "default")
         # .config("spark.spark.clickhouse.write.compression.codec", "lz4")
         # .config("spark.clickhouse.read.compression.codec", "lz4")
         # .config("spark.clickhouse.write.format", "arrow")
         #    .config("spark.clickhouse.write.distributed.convertLocal", "true") 
         #    .config("spark.clickhouse.write.repartitionNum", "1") 
         #.config("spark.clickhouse.write.maxRetry", "1000")
         #    .config("spark.clickhouse.write.repartitionStrictly", "true") 
         #    .config("spark.clickhouse.write.distributed.useClusterNodes", "false") 
         # .config("spark.clickhouse.write.batchSize", "10000000")
         # .config("spark.sql.catalog.clickhouse.socket_timeout", "600000000")
         # .config("spark.sql.catalog.clickhouse.connection_timeout", "600000000")
         # .config("spark.sql.catalog.clickhouse.query_timeout", "600000000")
         # .config("spark.clickhouse.options.socket_timeout", "600000000")
         # .config("spark.clickhouse.options.connection_timeout", "600000000")
         # .config("spark.clickhouse.options.query_timeout", "600000000")
         .config("spark.executor.memory", f"{ram}g")
         #.config("spark.executor.cores", "5")
         .config("spark.driver.maxResultSize", f"{ram}g")
         .config("spark.driver.memory", f"{ram}g")
         .config("spark.executor.memoryOverhead", f"{ram}g")
         #.config("spark.sql.debug.maxToStringFields", "100000")
         .getOrCreate()
         )
# spark.conf.set("spark.sql.catalog.clickhouse", "xenon.clickhouse.ClickHouseCatalog")
# spark.conf.set("spark.sql.catalog.clickhouse.host", "127.0.0.1")
# spark.conf.set("spark.sql.catalog.clickhouse.protocol", "http")
# spark.conf.set("spark.sql.catalog.clickhouse.http_port", "8123")
# spark.conf.set("spark.sql.catalog.clickhouse.user", "default")
# spark.conf.set("spark.sql.catalog.clickhouse.password", "")
# spark.conf.set("spark.sql.catalog.clickhouse.database", "default")
spark.sql("use clickhouse")

In [4]:
import findspark
findspark.init()
from pyspark import SparkConf, SparkContext, SQLContext
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error
import pandas as pd
import os

# Пакеты для подключения к ClickHouse
packages = [
    "com.github.housepower:clickhouse-spark-runtime-3.4_2.12:0.7.3",
    "com.clickhouse:clickhouse-jdbc:0.6.0-patch4",
    "com.clickhouse:clickhouse-http-client:0.6.0-patch4",
    "org.apache.httpcomponents.client5:httpclient5:5.3.1",
    "com.github.housepower:clickhouse-native-jdbc:2.7.1"
]

ram = 12
cpu = 12 * 3

# Инициализация SparkSession
appName = "Connect To ClickHouse via PySpark"
spark = (SparkSession.builder
         .appName(appName)
         .config("spark.jars.packages", ','.join(packages))
         .config("spark.sql.catalog.clickhouse", "xenon.clickhouse.ClickHouseCatalog")
         .config("spark.sql.catalog.clickhouse.host", "34.118.21.172")
         .config("spark.sql.catalog.clickhouse.protocol", "http")
         .config("spark.sql.catalog.clickhouse.http_port", "8123")
         .config("spark.sql.catalog.clickhouse.user", "default")
         .config("spark.sql.catalog.clickhouse.password", "1234")
         .config("spark.sql.catalog.clickhouse.database", "default")
         .config("spark.executor.memory", f"{ram}g")
         .config("spark.driver.maxResultSize", f"{ram}g")
         .config("spark.driver.memory", f"{ram}g")
         .config("spark.executor.memoryOverhead", f"{ram}g")
         .config("spark.driver.extraClassPath", "/path/to/clickhouse-jdbc-0.6.0-patch4.jar")
         .config("spark.executor.extraClassPath", "/path/to/clickhouse-jdbc-0.6.0-patch4.jar")
         .getOrCreate()
         )

# Загрузка данных из ClickHouse
df = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:clickhouse://34.118.21.172:8123/default") \
    .option("dbtable", "Weather_moscow_archive_updated") \
    .option("user", "default") \
    .option("password", "1234") \
    .option("driver", "com.clickhouse.jdbc.ClickHouseDriver") \
    .load()

# Преобразование данных в Pandas DataFrame для обработки XGBoost
pandas_df = df.toPandas()

# Предобработка данных
pandas_df['Local time in Moscow'] = pd.to_datetime(pandas_df['Local time in Moscow'])
pandas_df['T'] = pandas_df['T'].interpolate(method='linear')

# Разделение данных на тренировочные и тестовые
train_df = pandas_df[pandas_df['Local time in Moscow'] < '2023-01-01']
test_df = pandas_df[pandas_df['Local time in Moscow'] >= '2023-01-01']

# Извлечение фичей даты
def extract_date_features(df):
    df['year'] = df['Local time in Moscow'].dt.year
    df['month'] = df['Local time in Moscow'].dt.month
    df['day'] = df['Local time in Moscow'].dt.day
    df['hour'] = df['Local time in Moscow'].dt.hour
    return df

train_df = extract_date_features(train_df)
test_df = extract_date_features(test_df)

X_train = train_df[['year', 'month', 'day', 'hour']]
y_train = train_df['T']
X_test = test_df[['year', 'month', 'day', 'hour']]
y_test = test_df['T']

# Преобразование данных в формат DMatrix для XGBoost
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Параметры модели XGBoost
params = {
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse',
    'max_depth': 6,
    'eta': 0.1,
    'subsample': 0.8,
    'colsample_bytree': 0.8
}

# Обучение модели
bst = xgb.train(params, dtrain, num_boost_round=100)

# Прогнозирование
y_pred = bst.predict(dtest)

# Оценка модели
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'Mean Absolute Error: {mae}')

# Визуализация результатов
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.plot(test_df['Local time in Moscow'], y_test, label='Actual temperature')
plt.plot(test_df['Local time in Moscow'], y_pred, color='red', label='Predicted temperature')
plt.title('Temperature Forecast using XGBoost')
plt.xlabel('Date')
plt.ylabel('Temperature')
plt.legend()
plt.show()

# Остановка SparkSession
spark.stop()
